# Adiabatic sweep (with `QuantumObjectEvolution`)

Li-Xun Cai  
2025-07-10

Inspirations taken from [the QuTiP
tutorial](https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v5/lectures/Lecture-8-Adiabatic-quantum-computing.ipynb)
by J. R. Johansson.

This tutorial mainly demonstrates the use of
[`QuantumObjectEvolution`](https://qutip.org/QuantumToolbox.jl/stable/resources/api#QuantumToolbox.QuantumObjectEvolution).

## Introduction

The quantum adiabatic theorem is a fundamental principle in quantum
mechanics that describes how quantum systems evolve when subjected to
time-dependent conditions. This theorem states that if a quantum system
is initialized in an eigenstate (typically the ground state) of its
initial Hamiltonian, and if the Hamiltonian changes sufficiently slow,
the system will remain in the corresponding eigenstate of the evolving
Hamiltonian throughout the process.

For this theorem to hold, certain conditions must be satisfied: the
system must evolve slow enough compared to the energy gap between the
relevant eigenstate and other energy levels, and this gap must remain
non-zero throughout the evolution.

Here, we will demonstrate the well-known application of the adiabatic
theorem in quantum computing called $\text{\emph{adiabatic sweep}}$.
This is a method for preparing some desired quantum state by slowly
evolving a quantum system with a time-dependent Hamiltonian.
Essentially, the adiabatic theorem allows us to prepare the ground state
of the final Hamiltonian $H_1$ from the ground state of the initial
Hamiltonian $H_0$.

## Model

We consider a chain of $N$-identical spin-$1/2$ particles to study their
spin dynamics and set our interest in finding the ground state of the
condition that the spin chain has some random gap, leading to the random
magnitude $g_i$ of $\hat{\sigma}^i_x \hat{\sigma}^{i+1}_x$ interaction
with the neighboring spin.

Initially, we prepare the system such that the spins are free from
interaction with each other and are all in the ground state, i.e.,

$$H_0 = \sum_i^N \frac{\varepsilon_0}{2} \hat{\sigma}^i_z,$$
$$|\psi(0)\rangle = \bigotimes_{i=1}^N |g\rangle$$

Then, gradually, the Hamiltonian evolves to:

$$H_1 = \sum_{i=1}^N \frac{\varepsilon_0}{2} \hat{\sigma}^i_z + \sum_{i=1}^{N-1} g_i \hat{\sigma}^i_x \hat{\sigma}^{i+1}_x,$$

whose ground state are desired. By gradual change, we are subject to the
simplest form of adiabatic sweep, i.e.,

$$H(t,T) = H_0 * (1-t/T) + H_1 * t/T,$$

where the parameter $T$ determines how slow the Hamiltonian changes in
time.

## Code demonstration

In [1]:
using QuantumToolbox
using CairoMakie

CairoMakie.activate!(type = "svg")

In [1]:
N = 8 # number of spins
ε0 = 1 # energy gap of the spins

gs = rand(N-1) # use rand function for the random interaction strengths

7-element Vector{Float64}:
 0.3072294803873079
 0.6780554758187093
 0.33426139237525543
 0.3731132315583101
 0.04358533386148944
 0.4333677891055099
 0.020876113339115987

In [1]:
H0 = sum(1:N) do i
    ε0/2 * multisite_operator(Val(N), i=>sigmaz())
end

ψ0 = kron(fill(basis(2,1), N)...)
print(H0)


Quantum Object:   type=Operator()   dims=[2, 2, 2, 2, 2, 2, 2, 2]   size=(256, 256)   ishermitian=true
256×256 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 186 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀

In [1]:
Hint = sum(1:N-1) do i
    gs[i] * multisite_operator(Val(N), i=>sigmax(), i+1=>sigmax())
end

H1 = H0 + Hint

print(H1)


Quantum Object:   type=Operator()   dims=[2, 2, 2, 2, 2, 2, 2, 2]   size=(256, 256)   ishermitian=true
256×256 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 1978 stored entries:
⎡⠿⣧⣷⡈⠳⠄⠀⠘⠦⡀⠀⠀⠀⠀⠀⠳⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⡙⠻⣿⣿⣰⢦⡀⠀⠀⠈⠂⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠙⠆⠰⣞⠿⣧⣌⣦⠠⣄⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⣀⠀⠀⠈⠢⣽⣿⣿⣦⢈⠁⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠈⠣⡀⠀⠀⢦⡈⢛⢿⣷⡅⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎥
⎢⠀⠀⠈⠀⠀⠙⢅⠀⠁⠉⢿⣷⣧⡈⠻⠀⠀⠈⢢⡀⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠙⢄⠀⡉⠻⣵⣿⣴⠢⡀⠀⠀⠉⠀⠈⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡀⠀⠀⠀⠀⠀⠀⠀⠑⠛⠂⠰⡛⠛⣤⣼⡆⠠⣄⠀⠀⠀⠈⠓⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠑⢦⡀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠈⠲⠿⣿⢟⣦⣈⠀⠀⠀⠀⠀⠈⠓⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠲⡄⠀⠀⢦⡈⢻⣻⣾⠀⠀⠀⠀⠀⠀⠀⠈⠓⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢤⡀⠀⠀⠀⠀⠀⠀⠀⡿⣯⣧⡈⠳⠀⠀⠘⠦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢤⡀⠀⠀⠀⠀⠀⡉⠻⣵⣿⣶⠦⡀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠈⠳⢄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢤⡀⠀⠀⠀⠙⠂⠸⡟⠛⣤⣬⠆⠠⣤⢄⠀⠀⠀⠀⠀⠀⠀⠈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢄⡀⠀⣀⠀⠀⠈⠢⠟⣿⢟⣦⣈⠀⠑⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢄⠈⠣⡀⠀⠀⣦⡈⢻⢿⣷⣀⢀⠀⢑⣄⠀⠀⡀⠀⠀⎥
⎢⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⢘⢿⣷⣥⡈⠳⠀⠀⠈⢢⡀⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢄⢀⡁⠻⣿⣿⣟⠢⡀⠀⠀⠉⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⠙⠂⠻⡙⢻⣶⡽⠆⠰⣄⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠈⠳⠏⣿⣿⣦

Here, we define the time-dependent Hamiltonian with
`QuantumObjectEvolution`.

In [1]:
H = QuantumObjectEvolution((
    (H0, (p,t) -> 1 - t/p.T),
    (H1, (p,t) -> t/p.T),
))


Quantum Object Evo.:   type=Operator()   dims=[2, 2, 2, 2, 2, 2, 2, 2]   size=(256, 256)   ishermitian=true   isconstant=false
(ScalarOperator(0.0 + 0.0im) * MatrixOperator(256 × 256) + ScalarOperator(0.0 + 0.0im) * MatrixOperator(256 × 256))

We will show below the usage of field `p` in solving the eigen problems
and the dynamics.

In [1]:
function ψg(H)
    _, vecs = eigenstates(H)
    return vecs[1]
end

ψf_truth = ψg(H1) |> to_sparse
print(ψf_truth)


Quantum Object:   type=Ket()   dims=[2, 2, 2, 2, 2, 2, 2, 2]   size=(256,)
256-element SparseArrays.SparseVector{ComplexF64, Int64} with 128 stored entries:
  [1  ]  =  8.33035e-6+0.0im
  [4  ]  =  -0.000724454+0.0im
  [6  ]  =  0.000311122+0.0im
  [7  ]  =  -8.63494e-6-0.0im
  [10 ]  =  -0.00596127+0.0im
  [11 ]  =  0.000124508+0.0im
  [13 ]  =  -0.000312761+0.0im
         ⋮
  [241]  =  0.000238429+0.0im
  [244]  =  -0.0209472+0.0im
  [246]  =  0.0086202+0.0im
  [247]  =  -0.000230073+0.0im
  [250]  =  -0.182746+0.0im
  [251]  =  0.00381635+0.0im
  [253]  =  -0.00958841+0.0im
  [256]  =  0.880757+0.0im

We can see that the truthful ground state we are preparing is indeed
very complex.

For the adiabatic theorem to apply, we have to check for the gap between
the ground state and first excited state remaining non-zero throughout
the evolution.

In [1]:
T = 10
tlist = 0:0.1:T
eigs = Array{Float64}(undef, length(tlist), 2^N)

params = (T=T,)
for (idx, t) in enumerate(tlist)
    # passing `params` and `t` to `H` can yield the `QuantumObject` with `p = params` at time `t`
    vals, _ = eigenstates(H(params, t))
    eigs[idx,:] = vals
end

In [1]:
fig = Figure(size=(900, 400))
ax = Axis(fig[1,1], xticks = (0:0.25:1, ["$(t)T" for t in 0:0.25:1]))

for idx in 1:20 # only check for the lowest 20 eigenvalues
    color = (idx == 1) ? :magenta : (:gray,0.5)
    lines!(ax, range(0,1,length(tlist)), eigs[:,idx], label = string(idx), color = color)
end

fig

The plot shows that the gap is nonvanishing and thus validates the
evolution. So we proceed to check the expectation value dynamics of the
final Hamiltonian and the fidelity dynamics to the truthful ground state
throughout the evolution.

In [1]:
Tlist = 10 .^ (0:0.25:1.25)
results = map(Tlist) do T 
    tlist = range(0,T, 101)
    params = (T=T,)
    sesolve(H, ψ0, tlist, e_ops = [H1, ket2dm(ψf_truth)], params=params)
    # for solving dynamics, we can pass `params` to the keyword argument `params` for its reference
end;

Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)

In [1]:
fig = Figure(size=(900, 400))
axs = Axis.([fig[1,1], fig[1,2]])
axs[1].title = L"\langle H_f \rangle"
axs[1].xticks = (0:0.25:1, ["$(t)T" for t in 0:0.25:1])
axs[2].title = L"|\langle \psi_G^f |\psi(t)\rangle|^2"
axs[2].xticks = (0:0.25:1, ["$(t)T" for t in 0:0.25:1])

for ax_idx in 1:2, T_idx in 1:length(Tlist)
    T = Tlist[T_idx]
    exps = results[T_idx].expect
    tlist = range(0,1,101)
    lines!(axs[ax_idx], tlist, real(exps[ax_idx,:]), label = L"10^{%$(string(log10(T)))}")
end

Legend(fig[1,3], axs[1], L"T")

fig

As the plot showed, the fidelity between the prepared final state and
the truthful ground state reaches 1 as the total evolution time $T$
increases, showcasing the requirement of the adiabatic theorem that the
change has to be gradual.

## Version Information

In [1]:
QuantumToolbox.versioninfo()


 QuantumToolbox.jl: Quantum Toolbox in Julia
≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡
Copyright © QuTiP team 2022 and later.
Current admin team:
    Alberto Mercurio and Yi-Te Huang

Package information:
Julia              Ver. 1.11.5
QuantumToolbox     Ver. 0.31.1
SciMLOperators     Ver. 0.3.13
LinearSolve        Ver. 3.18.2
OrdinaryDiffEqCore Ver. 1.26.2

System information:
OS       : Linux (x86_64-linux-gnu)
CPU      : 4 × AMD EPYC 7763 64-Core Processor
Memory   : 15.621 GB
WORD_SIZE: 64
LIBM     : libopenlibm
LLVM     : libLLVM-16.0.6 (ORCJIT, znver3)
BLAS     : libopenblas64_.so (ilp64)
Threads  : 4 (on 4 virtual cores)
